# 深層学習特論レポート

"Association of genomic subtypes of lower-grade gliomas with shape features automatically
extracted by a deep learning algorithm"

- 氏名：楠 湧夢
- 学生番号：19676114
- paper with codeの分類：Brain Tumor Segmentation 
- 論文URL：https://arxiv.org/pdf/1906.03720v1.pdf
- GitHubのURL：https://github.com/mateuszbuda/brain-segmentation-pytorch
- Deep Learning Libraryの枠組み：PyTorch

脳 MRI における異常セグメンテーションのための事前学習された重みを用いた医用画像セグメンテーションのためのバッチ正規化を用いた U-Net

![](https://pytorch.org/assets/images/unet_brain_mri.png)

バッチ正規化と ReLU 活性化機能を有する２つの畳み込み層と、符号化部に１つの Max プーリング層、
復号部に代わりにアップ畳み込み層を含む４レベルのブロックから構成されている。

各ブロックの畳み込みフィルタの数は、３２個、６４個、１２８個、２５６個である。

バットネック層には５１２個の畳み込みフィルタがある。

符号化層からは、復号化部の対応する層へのスキップ接続が行われる。

入力画像は、プレコントラストシーケンス、 FLAIR シーケンス、ポストコントラストシーケンスから
それぞれ3チャンネルの脳 MRI スライスである。

出力は、入力画像と同じサイズの異常領域の１チャンネル確率マップである。

事前学習モデルの入力画像は、3つのチャンネルを持ち、256x256ピクセルにリサイズされ、体積あたりのZスコアが正規化されている必要があります。

In [1]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/archive/master.zip" to C:\Users\izumu/.cache\torch\hub\master.zip
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/releases/download/v1.0/unet-e012d006.pt" to C:\Users\izumu/.cache\torch\hub\checkpoints\unet-e012d006.pt


サンプル画像をダウンロードして使ってみる

![入力](TCGA_CS_4944.png)

In [3]:
# Download an example image
import urllib
url, filename = ("https://github.com/mateuszbuda/brain-segmentation-pytorch/raw/master/assets/TCGA_CS_4944.png", "TCGA_CS_4944.png")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [4]:
import numpy as np
from PIL import Image
from torchvision import transforms

input_image = Image.open(filename)
m, s = np.mean(input_image, axis=(0, 1)), np.std(input_image, axis=(0, 1))
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

print(torch.round(output[0]))

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])


得られた出力を白黒画像として保存

![出力](TCGA_CS_4944_inference.jpg)

In [23]:
im = torch.round(output[0]).clone().numpy()
im = 255*im[0]
print(im)

pil_image_gray = Image.fromarray(im.astype(np.uint8))
print(pil_image_gray.mode)
pil_image_gray.save('TCGA_CS_4944_inference.jpg')

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
L


## モデルのサマリ

３チャネル 256×256 画像を入力に指定することで `torchsummary` を用いてサマリを確認

In [5]:
from torchsummary import summary

summary(model, (3,256,256))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 256, 256]             864
       BatchNorm2d-2         [-1, 32, 256, 256]              64
              ReLU-3         [-1, 32, 256, 256]               0
            Conv2d-4         [-1, 32, 256, 256]           9,216
       BatchNorm2d-5         [-1, 32, 256, 256]              64
              ReLU-6         [-1, 32, 256, 256]               0
         MaxPool2d-7         [-1, 32, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          18,432
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
           Conv2d-11         [-1, 64, 128, 128]          36,864
      BatchNorm2d-12         [-1, 64, 128, 128]             128
             ReLU-13         [-1, 64, 128, 128]               0
        MaxPool2d-14           [-1, 64,